<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 9: Natural Language Processing with TensorFlow – Sentiment Analysis**
Bab ini membahas analisis sentimen menggunakan TensorFlow, dimulai dari preprocessing teks hingga implementasi model LSTM dengan word embeddings. Dataset yang digunakan adalah ulasan video game dari Amazon, dengan tujuan mengklasifikasikan ulasan menjadi positif (label 1) atau negatif (label 0).



# **Langkah-langkah Utama:**
# **1. Eksplorasi dan Preprocessing Data**
---
* Dataset: Video_Games_5.json.gz dari Amazon.
* Preprocessing meliputi:
* Mengonversi teks menjadi huruf kecil.
* Menghapus tanda baca, angka, dan stopwords (kecuali "not" dan "no").
* Lemmatization menggunakan WordNetLemmatizer dari NLTK.
* Tokenization menjadi kata-kata individual.

# **2. Persiapan Data untuk Model**
---
* Split Data: Training, validation, test dengan stratified sampling untuk menangani ketidakseimbangan kelas.
* Analisis Vocabulary: Menghitung frekuensi kata dan menentukan ukuran vocabulary (n_vocab ≈ 11.800).
* Analisis Panjang Sequence: Membagi ulasan menjadi kategori pendek (<5 kata), sedang (5–15 kata), dan panjang (>15 kata).
* Tokenization dengan Keras: Mengonversi kata menjadi ID numerik menggunakan Tokenizer.

# **3. Pipeline Data dengan TensorFlow**
---
* Bucket_by_sequence_length: Mengelompokkan ulasan dengan panjang seragam untuk efisiensi padding.
* Menggunakan tf.RaggedTensor untuk menangani sequence dengan panjang variabel.
* Pipeline akhir menghasilkan batch data siap untuk training.



# **4. Model LSTM untuk Analisis Sentimen**
---
Arsitektur Model:
* Masking Layer untuk mengabaikan padding.
* One-hot Encoding (dalam model awal) atau Embedding Layer (dalam model final).
* LSTM Layer dengan 128 unit.
* Dense Layer dengan 512 unit + ReLU.
* Dropout Layer (0.5).
* Output Layer dengan 1 unit + sigmoid.

# **5. Training dan Evaluasi**
---
* Loss: binary_crossentropy.
* Optimizer: Adam.
* Class Weight: Memberi bobot lebih pada kelas minoritas (ulasan negatif).
* Callback: CSVLogger, ReduceLROnPlateau, EarlyStopping.
* Hasil Akhir: Akurasi ≈ 80–81% pada test set.

# **6. Peningkatan dengan Word Embeddings**
---
* Mengganti one-hot encoding dengan Embedding Layer (tf.keras.layers.Embedding).
* Embedding Layer belajar representasi vektor kata selama training.
* Hasil: Peningkatan akurasi menjadi ≈ 81.1% pada test set.

# **7. Validasi Manual Prediksi**
---
* Melihat ulasan paling negatif dan paling positif berdasarkan prediksi model.
* Kesimpulan: Model memberikan prediksi yang sesuai dengan konteks ulasan.

# **Kode Utama yang Direproduksi:**
---
### **1. Preprocessing Teks**

In [ ]:
def clean_text(doc):
    doc = doc.lower()
    doc = doc.replace("n\'t ", ' not ')
    doc = re.sub(r"(?:\\'ll |\\'re |\\'d |\\'ve )", " ", doc)
    doc = re.sub(r"/d+", "", doc)
    tokens = [w for w in word_tokenize(doc) if w not in EN_STOPWORDS and w not in string.punctuation]
    pos_tags = nltk.pos_tag(tokens)
    clean_text = [lemmatizer.lemmatize(w, pos=p[0].lower()) if p[0]=='N' or p[0]=='V' else w for (w, p) in pos_tags]
    return clean_text

### **2. Tokenization dengan Keras**

In [ ]:
tokenizer = Tokenizer(num_words=n_vocab, oov_token='unk', lower=False, filters='', split=' ', char_level=False)
tokenizer.fit_on_texts(tr_x.tolist())
tr_x = tokenizer.texts_to_sequences(tr_x.tolist())

### **3. Pipeline Data dengan Bucketing**

In [ ]:
def get_tf_pipeline(text_seq, labels, batch_size=64, bucket_boundaries=[5,15], max_length=50, shuffle=False):
    data_seq = [[b]+a for a,b in zip(text_seq, labels)]
    tf_data = tf.ragged.constant(data_seq)[:, :max_length]
    text_ds = tf.data.Dataset.from_tensor_slices(tf_data)
    bucket_fn = tf.data.experimental.bucket_by_sequence_length(
        lambda x: tf.cast(tf.shape(x)[0], 'int32'),
        bucket_boundaries=bucket_boundaries,
        bucket_batch_sizes=[batch_size]*3,
        padding_values=0,
        pad_to_bucket_boundary=False
    )
    text_ds = text_ds.map(lambda x: x).apply(bucket_fn)
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=10*batch_size)
    text_ds = text_ds.map(lambda x: (x[:,1:], x[:,0]))
    return text_ds

### **4. Model LSTM dengan Embedding**

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=n_vocab, output_dim=128, mask_zero=True, input_shape=(None,)),
    tf.keras.layers.LSTM(128, return_state=False, return_sequences=False),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])